## Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import locale
import re
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import mode, skew, kurtosis
from dotenv import load_dotenv
import sys
import os

## Variável de Ambiente

In [2]:
# Carregar as variáveis do arquivo .env
load_dotenv()

# Obter o caminho do arquivo do .env
excel_file = os.getenv("EXCEL_FILE_PATH")

# Testar o caminho
if os.path.exists(excel_file):
    print("Arquivo encontrado!")
else:
    print("Arquivo não encontrado!")

Arquivo não encontrado!


## Visualização inicial do DataFrame

In [3]:
# Leitura do arquivo Excel e seleção das colunas desejadas
df = pd.read_excel(excel_file, sheet_name='Planilha1', usecols=['Data', 'Valor', 'Status', 'Operador', 'Vendedor'])

# Exibição o número de linhas e colunas do DataFrame
print("Número de Linhas e Colunas")
print(df.shape)
print("------------")

# Exibir as primeiras linhas do DataFrame
print("Primeiras Linhas do DataFrame")
print(df.head())
print("------------")

# Exibir os tipos de dados ajustados
print("Tipos de Dados")
print(df.dtypes)

OSError: [Errno 22] Invalid argument: 'r"C:\\Users\\rodri\\OneDrive\\Sincronização\\Documentos\\GitHub\\esquadrao_loja\\dados\\Export_Vendas_Sintetico.xlsx"'

## Ajuste dos dados

In [ ]:
# Renomear as colunas
df.rename(columns={
    'Data': 'data',
    'Valor': 'valor',
    'Status': 'status',
    'Operador': 'operador',
    'Vendedor': 'vendedor'
}, inplace=True)

# Ajustar as variáveis da coluna status
df['status'] = df['status'].str.capitalize()

# Definir o locale para Português do Brasil (pt_BR)
locale.setlocale(locale.LC_NUMERIC, 'pt_BR.UTF-8')

# Função para converter valores
def convert_to_float(value):
    if isinstance(value, str):
        value = value.replace('.', '').replace(',', '.')
    return float(value)

# Aplicar a função de conversão na coluna 'valor' para float aplicando
df['valor'] = df['valor'].apply(convert_to_float)

# Converter a coluna 'data' para Timestamp
try:
    df['data'] = pd.to_datetime(df['data'])
except ValueError as e:
    print(f"Não foi possível converter a coluna 'data' para Timestamp: {e}")

# Truncar a coluna 'status' para 10 caracteres
df['status'] = df['status'].apply(lambda x: x[:10])

# Remover linhas onde o status é 'Cancelada' ou 'Inutilizada'
df = df[~df['status'].isin(['Cancelada', 'Inutilizad'])]

# Exibir as primeiras linhas do DataFrame após a conversão
print("Primeiras Linhas do DataFrame - compras devidamente efetivadas")
print(df.head())
print("------------")

# Exibir os tipos de dados ajustados
print("Tipos de Dados")
print(df.dtypes)

## Checagem dos Dados

In [ ]:
# visualizar o DataFrame

# Ajustar as opções de exibição do pandas
pd.options.display.float_format = '{:.2f}'.format

# Exibir o DataFrame completo
df

In [ ]:
# Exibição o número de linhas e colunas do DataFrame
print("Número de Linhas e Colunas")
print(f"Número de Linhas e Colunas: {df.shape[0]} linhas, {df.shape[1]} colunas")

# Exibição os nomes das colunas do DataFrame
print("\nNomes das Colunas")
print(df.columns.to_list())

# Exibição informações gerais sobre o DataFrame
print("\nInformações Gerais")
df.info()

# Exibição do número de valores vazios (nulos) em cada coluna
print("\nChecagem de Valores Vazios")
print(df.isnull().sum())

# Exibição do número de linhas duplicadas no DataFrame
print("\nNúmero de Linhas Duplicadas")
print(df.duplicated().sum())

In [ ]:
# Exibe o número de valores únicos em cada coluna do DataFrame
print(df.nunique())

print("------------")

# Extrai os valores únicos da coluna 'operador' e armazena em uma variável
operadores = df["operador"].unique()

# Imprime a lista de operadores únicos, formatada como uma string
print(f"Os operadores são: {', '.join(map(str, operadores))}")

print("------------")

# Extrai os valores únicos da coluna 'vendedor' e armazena em uma variável
vendedores = df["vendedor"].unique()

# Imprime a lista de vendedores únicos, formatada como uma string
print(f"Os vendedores são: {', '.join(map(str, vendedores))}")

print("------------")

# Extrai os valores únicos da coluna 'status' para conferir se o df apresenta apenas as compras regulares
status_regular = df["status"].unique()

# Imprime a lista de operadores únicos, formatada como uma string
print(f"O status é apenas: {', '.join(map(str, status_regular))}")

## Análise Exploratória

### Dados Estatísticos

In [ ]:
# Exibição dos dados estatísticos das colunas numéricas com duas casas decimais
with pd.option_context("float_format", "{:.2f}".format):
    display(df.describe())

In [ ]:
# Função para calcular as estatísticas
def calcular_estatisticas(amostra):
    moda_result = mode(amostra, keepdims=True)  # Garantir que a moda é retornada como array
    estatisticas = {
        "Média": np.mean(amostra),
        "Mediana": np.median(amostra),
        "Moda": moda_result.mode[0] if len(moda_result.mode) > 0 else np.nan,
        "Variância": np.var(amostra, ddof=1),
        "Desvio Padrão": np.std(amostra, ddof=1),
        "Assimetria": skew(amostra),
        "Curtose": kurtosis(amostra, fisher=False),  # fisher=False retorna a curtose de momento
        "1º Quartil": np.percentile(amostra, 25),
        "2º Quartil (Mediana)": np.percentile(amostra, 50),
        "3º Quartil": np.percentile(amostra, 75)
    }
    return estatisticas

# Aplicar a função
estatisticas_df_valor = calcular_estatisticas(df['valor'].to_numpy())

# Exibir os resultados
print("Estatísticas para a coluna 'valor' do DataFrame:")
for chave, valor in estatisticas_df_valor.items():
    print(f"{chave}: {valor:.2f}")

In [ ]:
# Adicionar o caminho da pasta 'modules' ao Python
module_path = os.path.abspath("modules")
if module_path not in sys.path:
    sys.path.append(module_path)

from estatisticas import calcular_estatisticas

# Aplicar a função
estatisticas_df_valor = calcular_estatisticas(df['valor'].to_numpy())

# Exibir os resultados
print("Estatísticas para a coluna 'valor' do DataFrame:")
for chave, valor in estatisticas_df_valor.items():
    print(f"{chave}: {valor:.2f}")

### Box Plot da coluna de valor da compra

In [ ]:
# Configurar o estilo do gráfico
sns.set(style="whitegrid")

# Criar a figura e os subplots
plt.figure(figsize=(12, 6))

# Subplot para 'valor'
plt.subplot(1, 2, 1)

# Criar o boxplot
sns.boxplot(y=df['valor'], color='skyblue', showmeans=True)

# Adicionar o título
plt.title('Boxplot do Valor')

# Calcular medidas resumo
mean = df['valor'].mean()
median = df['valor'].median()
min_value = df['valor'].min()
max_value = df['valor'].max()
std_dev = df['valor'].std()

# Adicionar linhas horizontais para média, mediana, mínimo, máximo e desvio padrão
plt.axhline(mean, color='red', linestyle='--', label=f'Média: {mean:.2f}')
plt.axhline(median, color='green', linestyle='--', label=f'Mediana: {median:.2f}')
plt.axhline(min_value, color='purple', linestyle=':', label=f'Mínimo: {min_value:.2f}')
plt.axhline(max_value, color='orange', linestyle=':', label=f'Máximo: {max_value:.2f}')
plt.axhline(mean + std_dev, color='blue', linestyle='--', label=f'Média + Desvio Padrão: {mean + std_dev:.2f}')
plt.axhline(mean - std_dev, color='blue', linestyle='--', label=f'Média - Desvio Padrão: {mean - std_dev:.2f}')

# Adicionar a legenda
plt.legend()

# Ajustar layout e exibir o gráfico
plt.tight_layout()
plt.show()

### Verificação das compras acima do limite superior (outliers)

In [ ]:
# Calcular Q1 (Primeiro Quartil) e Q3 (Terceiro Quartil)
q1 = np.percentile(df['valor'], 25)
q3 = np.percentile(df['valor'], 75)

# Calcular o Intervalo Interquartílico (IQR)
iqr = q3 - q1

# Calcular o Limite Superior
limite_superior = q3 + 1.5 * iqr

print(f"Primeiro Quartil (Q1): {q1}")
print(f"Terceiro Quartil (Q3): {q3}")
print(f"Intervalo Interquartílico (IQR): {iqr}")
print(f"Limite Superior: {limite_superior}")

print("------------")

# Filtrar os dados que estão acima do limite superior
outlier = df[df['valor'] > limite_superior]

print("Dados acima do limite superior:")
print(outlier)

### Verificação de desempenho por operador

In [ ]:
# Calcular o desempenho total por operador
desempenho_por_operador = df.groupby('operador')['valor'].sum().reset_index()

# Criar o gráfico de barras
plt.figure(figsize=(10, 6))  # Aumentar o tamanho da figura para dar mais espaço
ax = sns.barplot(x='operador', y='valor', data=desempenho_por_operador, palette='viridis')

# Adicionar título e rótulos dos eixos
plt.title('Desempenho por Operador')
plt.xlabel('Operador')
plt.ylabel('Total de Valor')
plt.xticks(rotation=45, ha='right')  # Ajustar a rotação e o alinhamento dos rótulos dos eixos X

# Ajustar o limite do eixo y para garantir que todos os rótulos sejam visíveis
max_valor = desempenho_por_operador['valor'].max()
plt.ylim(0, max_valor * 1.1)  # Define o limite superior do eixo y um pouco acima do valor máximo

# Adicionar rótulos de dados
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{height:,.2f}', 
                (p.get_x() + p.get_width() / 2., height), 
                ha='center', va='bottom', 
                xytext=(0, 3), 
                textcoords='offset points')

# Ajustar layout e exibir o gráfico
plt.tight_layout()
plt.show()

### Verificação de desempenho por vendedor

In [ ]:
# Calcular o desempenho total por vendedor
desempenho_por_vendedor = df.groupby('vendedor')['valor'].sum().reset_index()

# Criar o gráfico de barras
plt.figure(figsize=(12, 6))  # Ajustar o tamanho da figura
ax = sns.barplot(x='vendedor', y='valor', data=desempenho_por_vendedor, palette='viridis')

# Adicionar título e rótulos dos eixos
plt.title('Desempenho por Vendedor')
plt.xlabel('Vendedor')
plt.ylabel('Total de Valor')
plt.xticks(rotation=45, ha='right')  # Ajustar a rotação e o alinhamento dos rótulos dos eixos X

# Ajustar o limite do eixo y para garantir que todos os rótulos sejam visíveis
max_valor = desempenho_por_vendedor['valor'].max()
plt.ylim(0, max_valor * 1.1)  # Define o limite superior do eixo y um pouco acima do valor máximo

# Adicionar rótulos de dados
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{height:,.2f}', 
                (p.get_x() + p.get_width() / 2., height), 
                ha='center', va='bottom', 
                xytext=(0, 3), 
                textcoords='offset points')

# Ajustar layout e exibir o gráfico
plt.tight_layout()
plt.show()